In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sales = glueContext.create_dynamic_frame.from_catalog(
                 database="s3_database",
                 table_name="sales")
print ("Count: ", sales.count())
sales.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count:  2823
root
|-- ordernumber: long
|-- quantityordered: long
|-- priceeach: double
|-- orderlinenumber: long
|-- sales: double
|-- orderdate: string
|-- status: string
|-- qtr_id: long
|-- month_id: long
|-- year_id: long
|-- productline: string
|-- msrp: long
|-- productcode: string
|-- dealsize: string
|-- customerid: long

## ApplyMapping

In [8]:
new_dyf = ApplyMapping.apply(frame = sales, mappings = [('ordernumber','long','ordernumber','double'),])
new_dyf.toDF().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|ordernumber|
+-----------+
|    10107.0|
|    10121.0|
|    10134.0|
|    10145.0|
|    10159.0|
+-----------+
only showing top 5 rows

## DropFields

In [16]:
new_dyf = DropFields.apply(sales,paths=['dealsize','customerid'])
new_dyf.toDF().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+
|ordernumber|quantityordered|priceeach|orderlinenumber|  sales|      orderdate| status|qtr_id|month_id|year_id|productline|msrp|productcode|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped|     1|       2|   2003|Motorcycles|  95|   S10_1678|
|      10121|             34|    81.35|              5| 2765.9|  5/7/2003 0:00|Shipped|     2|       5|   2003|Motorcycles|  95|   S10_1678|
|      10134|             41|    94.74|              2|3884.34|  7/1/2003 0:00|Shipped|     3|       7|   2003|Motorcycles|  95|   S10_1678|
|      10145|             45|    83.26|              6| 3746.7| 8/25/2003 0:00|Shipped|     3|       8|   2003|Motorcycles|  95|   S10_1678|
|      10159|

## DropNullFields

In [18]:
new_dyf = DropNullFields.apply(frame=sales)
new_dyf.toDF().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

null_fields []
2823

## Filter

In [28]:
new_dyf = Filter.apply(frame=sales, f = lambda x: x['priceeach'] < 30)
new_dyf.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------+-------+------------+------+----+---------------+-----------+----------+-----------+---------------+----------+--------+---------+--------+
|orderlinenumber|  sales|year_id| productline|qtr_id|msrp|quantityordered|productcode|    status|ordernumber|      orderdate|customerid|dealsize|priceeach|month_id|
+---------------+-------+-------+------------+------+----+---------------+-----------+----------+-----------+---------------+----------+--------+---------+--------+
|             13|1224.67|   2004|Vintage Cars|     4|  62|             41|   S18_2957|   Shipped|      10322| 11/4/2004 0:00|       928|   Small|    29.87|      11|
|              5|1007.14|   2003|Vintage Cars|     4|  33|             37|   S24_1937|   Shipped|      10162|10/18/2003 0:00|      1544|   Small|    27.22|      10|
|             10| 759.46|   2003|Vintage Cars|     4|  33|             26|   S24_1937|   Shipped|      10193|11/21/2003 0:00|      1547|   Small|    29.21|      11|
|         

## SelectFields

In [30]:
new_dyf = SelectFields.apply(frame=sales, paths=['month_id'])
new_dyf.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|month_id|
+--------+
|       2|
|       5|
|       7|
|       8|
|      10|
|      10|
|      11|
|      11|
|      12|
|       1|
|       2|
|       4|
|       5|
|       6|
|       7|
|       8|
|       9|
|      10|
|      11|
|      11|
+--------+
only showing top 20 rows